In [2]:
import librosa
import librosa.display
import pandas as pd
%matplotlib inline
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
tqdm.pandas()
from matplotlib import rcParams
rcParams['axes.titlepad'] = 20
import os
from os import path
import pickle
import math

#Clustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

#librosa
import IPython.display as ipd

# Get list of songs

In [3]:
directory = "../Google Drive/datahack2019/taylor/"
sub_dirs = [x[0] for x in os.walk(directory)]
song_files = []
for sub_dir in sub_dirs:
    for r, d, f in os.walk(sub_dir):
        for file in f:
            if '.mp3' in file:
                song_files.append(os.path.join(r, file))
                
song_files = set(song_files)

# Windows by 10 secs

In [4]:
size_of_window = 10 #in seconds

window_items_list = []
for ind,song in enumerate(tqdm(song_files)):
    
    y, sr = librosa.load(song)
    
    n_fft = 2048
    hop_length = 512
    n_mels = 128
    S = librosa.feature.melspectrogram(y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    
    windows_of_melspec_idx = []
    melspec_data_of_windows = []
    for i in range(int(librosa.core.get_duration(y)//size_of_window)):
        windows_of_melspec_idx.append(slice(*librosa.time_to_frames([i*size_of_window, i*size_of_window+ size_of_window], sr=sr, hop_length=hop_length)))
    #windows_of_melspec_idx.append(slice(librosa.time_to_frames(i*10+ 10),S_DB.shape[1])) # add last window even if shorter than 10 secs, will need padding
    for slice_idx in windows_of_melspec_idx:
        windows_item = {}
        windows_item["song"] = song
        windows_item["window_idx"] = slice_idx
        w_d = S_DB[:,slice_idx][:,:math.floor((22050/512)*size_of_window)]
        windows_item["data"] = w_d
        melspec_data_of_windows.append(w_d)
        window_items_list.append(windows_item)
        
        sec_start = librosa.core.frames_to_time(slice_idx.start)
        loc_start = sec_start * sr
        loc_end = (sec_start + size_of_window) * sr
        windows_item["loc_start"] = loc_start
        windows_item["loc_end"] = loc_end
        windows_item["song_snippet"] = y[int(loc_start):int(loc_end)]
        windows_item["sr"] = sr
        
    #if ind == 10:
    #    break


100%|██████████| 30/30 [05:28<00:00, 10.97s/it]


In [5]:
windows_df = pd.DataFrame(window_items_list)

In [6]:
del window_items_list

# Windows by onsets

In [17]:
taylor_df_path = "../Google Drive/datahack2019/VideoThat/sandbox/data/taylor_df.pkl"
with open(taylor_df_path, 'rb') as handle:
    taylor_df = pickle.load(handle)

In [49]:
def get_dir_path(name):
    res = [x for x in sub_dirs if x.endswith(name)]
    if len(res) == 1:
        for r, d, f in os.walk(res[0]):
            song_res = []
            for file in f:
                if '.mp3' in file:
                    song_res.append(os.path.join(r, file))
    if len(song_res) == 1:
        return song_res[0]
    else:
        print("error , " + len(song_res))
        
taylor_df["song_path"] = taylor_df.name.apply(get_dir_path)

In [55]:
taylor_df.head()

,name,song_wave,scene_start_times_sec,onset_times_sec,song_path
0,Taylor Swift - White Horse-D1Xr-JFLxik,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 0.667 2 3.587 3...,"[0.6965986394557823, 1.3467573696145125, 1.648...",/Users/ydaihes/Google Drive/datahack2019/taylo...
1,Taylor Swift - The Story Of Us-nN6VR92V70M,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 0.500 2 1.417 3...,"[0.6501587301587302, 0.905578231292517, 1.0913...",/Users/ydaihes/Google Drive/datahack2019/taylo...
2,Taylor Swift - Delicate-tCXGJQYZ9JA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 3.045 2 6.715 3 ...,"[3.8312925170068026, 4.040272108843538, 4.2028...",/Users/ydaihes/Google Drive/datahack2019/taylo...
3,Taylor Swift - Style--CmadmM5cOk,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 7.424 2 9.259 3...,"[0.11609977324263039, 10.077460317460318, 10.4...",/Users/ydaihes/Google Drive/datahack2019/taylo...
4,Taylor Swift - ME! (feat. Brendon Urie of Pani...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 0.500 2 5.255 3 ...,"[17.29886621315193, 19.11002267573696, 19.6208...",/Users/ydaihes/Google Drive/datahack2019/taylo...


In [97]:
def pad_zeros(arr, expected_len, hight):
    return np.concatenate((arr, np.zeros((hight, expected_len - np.shape(arr)[1]))), axis=1)

In [329]:
window_items_list = []
for taylor_df_row in tqdm(list(taylor_df.itertuples())):
    y, sr = librosa.load(taylor_df_row.song_path)

    #print(taylor_df_row.song_path)
    
    n_fft = 2048
    hop_length = 512
    n_mels = 128
    S = librosa.feature.melspectrogram(y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)

    current_Secs = taylor_df_row.scene_start_times_sec
    windows_of_melspec_idx = []
    for i in range(0, len(current_Secs) - 1):
        windows_of_melspec_idx.append(slice(*librosa.time_to_frames([current_Secs[i], current_Secs[i+1]], sr=sr, hop_length=hop_length)))
    for slice_idx in windows_of_melspec_idx:
        windows_item = {}
        windows_item["song"] = taylor_df_row.song_path
        windows_item["window_idx"] = slice_idx
        w_d = S_DB[:,slice_idx]#[:,:math.floor((22050/512)*size_of_window)]
        windows_item["data"] = w_d

        loc_start = librosa.core.frames_to_time(slice_idx.start) * sr
        loc_end = librosa.core.frames_to_time(slice_idx.stop) * sr
        windows_item["loc_start"] = loc_start
        windows_item["loc_end"] = loc_end
        windows_item["song_snippet"] = y[int(loc_start):int(loc_end)]
        windows_item["sr"] = sr

        window_items_list.append(windows_item)

  0%|          | 0/30 [00:00<?, ?it/s]

/Users/ydaihes/Google Drive/datahack2019/taylor/Taylor Swift - White Horse-D1Xr-JFLxik/Taylor Swift - White Horse-D1Xr-JFLxik.mkv.mp3


  3%|▎         | 1/30 [00:09<04:39,  9.63s/it]

/Users/ydaihes/Google Drive/datahack2019/taylor/Taylor Swift - The Story Of Us-nN6VR92V70M/Taylor Swift - The Story Of Us-nN6VR92V70M.mkv.mp3


  7%|▋         | 2/30 [00:19<04:38,  9.95s/it]

...

 93%|█████████▎| 28/30 [05:40<00:24, 12.17s/it]

/Users/ydaihes/Google Drive/datahack2019/taylor/Taylor Swift - Back To December-QUwxKWT6m7U/Taylor Swift - Back To December-QUwxKWT6m7U.mkv.mp3


 97%|█████████▋| 29/30 [05:53<00:12, 12.18s/it]

/Users/ydaihes/Google Drive/datahack2019/taylor/Taylor Swift - Picture To Burn-yCMqcFAigRg/Taylor Swift - Picture To Burn-yCMqcFAigRg.mkv.mp3


100%|██████████| 30/30 [06:01<00:00, 12.06s/it]


In [331]:
sub_windows_df = windows_df[windows_df.data.apply(lambda x: np.shape(x)[1] < 647)]

In [332]:
len(sub_windows_df)

3709

In [333]:
sub_windows_df["padded_data"] = sub_windows_df.data.apply(lambda x: pad_zeros(x, 647, x.shape[0]))

/Users/ydaihes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [334]:
sub_windows_df["duration"] = windows_df.apply(lambda x: x.loc_end - x.loc_start, axis=1)

/Users/ydaihes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [335]:
def get_tempo(df_row):
    onset_env = librosa.onset.onset_strength(df_row.song_snippet)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=df_row.sr)


# Clustering

In [343]:
df_to_cluster = sub_windows_df[sub_windows_df.duration > 40000]

In [344]:
n_windows, n_hops, n_frames_in_window_size = np.array(list(df_to_cluster.padded_data)).shape
d2_train_dataset = np.array(list(df_to_cluster.padded_data)).reshape((n_windows,n_hops*n_frames_in_window_size))
d2_train_dataset.shape

(1147, 82816)

In [349]:
hyper_params = {
    "epsilon" : 2500,
    "min_samples" : 2,
    #"min_cluster_num" : 
}

db = DBSCAN(eps = hyper_params["epsilon"], min_samples = hyper_params["min_samples"], metric='euclidean',algorithm='brute').fit(d2_train_dataset)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_ 

if "DBSCAN_cluster" in df_to_cluster.columns:
        del df_to_cluster["DBSCAN_cluster"]
        
df_to_cluster.insert(len(df_to_cluster.columns), "DBSCAN_cluster", labels)
df_to_cluster.DBSCAN_cluster.value_counts()

 0     1031
-1       91
 4        4
 6        3
 5        3
 3        3
 10       2
 9        2
 8        2
 7        2
 2        2
 1        2
Name: DBSCAN_cluster, dtype: int64

In [350]:
df_to_cluster[df_to_cluster.DBSCAN_cluster == 4]#.iloc[1].song

,data,loc_end,loc_start,song,song_snippet,sr,window_idx,padded_data,duration,DBSCAN_cluster
428,"[[-35.12468, -30.685572, -30.445894, -31.22792...",2434048.0,2249216.0,/Users/ydaihes/Google Drive/datahack2019/taylo...,"[0.168077, 0.12736185, 0.10891143, 0.07502436,...",22050,"slice(4393, 4754, None)","[[-35.12467956542969, -30.685571670532227, -30...",184832.0,4
1163,"[[-18.43669, -18.825489, -18.933107, -20.33292...",4244992.0,4061696.0,/Users/ydaihes/Google Drive/datahack2019/taylo...,"[0.1454173, 0.18454534, 0.1534005, 0.22488262,...",22050,"slice(7933, 8291, None)","[[-18.436689376831055, -18.825489044189453, -1...",183296.0,4
1892,"[[-40.1942, -44.050034, -49.138733, -42.133026...",5322752.0,5140480.0,/Users/ydaihes/Google Drive/datahack2019/taylo...,"[0.08989131, 0.07891475, 0.024642197, -0.00739...",22050,"slice(10040, 10396, None)","[[-40.19419860839844, -44.05003356933594, -49....",182272.0,4
3191,"[[-37.726982, -35.099087, -41.611774, -38.9925...",1456640.0,1276416.0,/Users/ydaihes/Google Drive/datahack2019/taylo...,"[0.16875245, 0.16259749, 0.06459695, 0.0495665...",22050,"slice(2493, 2845, None)","[[-37.72698211669922, -35.09908676147461, -41....",180224.0,4


In [164]:
def play_window(df_row):
    return ipd.Audio(df_row.song_snippet, rate=df_row.sr)

In [342]:
play_window(windows_df.iloc[3714])

In [183]:
play_window(windows_df.iloc[97])